In [1]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters import * 

In [2]:
data = io.load('data/gauss4')

In [3]:
import warnings

warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

### Helper functions

#### view

In [4]:
def update_view(view, f):
    def _inner():
        view.show = filter_tree(view.tree, f)
    return _inner 

In [5]:
def view(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=update_view(v, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

#### reduce

In [6]:
def reduce_view(widget, tree, f):
    @debug.capture()
    def _inner():
        widget.ref = reduce_tree(tree, filter=f)
    return _inner

In [7]:
def reduce(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data.tree, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=reduce_view(v, data.tree, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

### Initial tree

In [8]:
tw = TreeWidget(data.tree)

In [9]:
v1, f1, p1 = view(tw, title='Initial Tree')

In [10]:
v2, f2, p2 = view(tw, attr='fitness', title='fitness')

### Details

In [11]:
dw = DataWidget(data=data)

In [12]:
details = DetailsView(data=dw)

In [13]:
p2 = SidePanel(title='details')
with p2:
    display(details)

In [14]:
fitness = view(tw, attr='fitness', title='fitness')

In [15]:
parent_fitness = view(tw, attr='parent_fitness', title='parent_fitness')

In [16]:
l1 = widgets.link((fitness[0], 'details'), (details, 'show'))

In [17]:
l2 = widgets.link((parent_fitness[0], 'details'), (details, 'show'))

In [18]:
parent_fitness[0].details = []

In [19]:
fitness[0].details = []

### Test custom widget

In [27]:
vbox = widgets.VBox([widgets.Text('A', disabled=True)])

In [28]:
display(vbox)

In [29]:
def on_selection_changed(change):
    vbox.value = change['new']
    
fitness[0].observe(on_selection_changed, names='details')

In [33]:
fitness[0].details

[45, 0]